In [1]:
!pip install replicate


In [2]:
!pip install python-dotenv

In [3]:
!pip install openai --upgrade



In [4]:
import replicate
from dotenv import load_dotenv
import openai
import os

load_dotenv()

# Initialize Replicate client with the API token from environment variables
replicate_api_token = os.getenv("REPLICATE_API_TOKEN")
replicate_client = replicate.Client(api_token=replicate_api_token)

# Initialize OpenAI client with the API key from environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = openai_api_key

In [5]:
import openai
from dotenv import load_dotenv
import os

# Load environment variables from a .env file
load_dotenv()

# Initialize OpenAI client with the API key from environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the input parameters for the OpenAI model using the updated API
response = openai.Completion.create(
    model="text-davinci-003",
    prompt="What is the capital of France?",
    max_tokens=50,
    temperature=0.7
)

# Print the result
print(response.choices[0].text.strip())


APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
